In [1]:
import arcpy
from arcgis.gis import *
#these next two support random string generation
from random import choice
from string import ascii_uppercase


arcpy.env.overwriteOutput = True

#Get the layer from the TOC
featurelayer = "ZoneTemplate"
zoneNameField = "displocname"
fields = ["displocname","routename","objectid","shape@"]

#Set an expression to optionally use as a whereclause
expression = "{} = 'DEN'".format(arcpy.AddFieldDelimiters(featurelayer, zoneNameField))

#create a table in scratch space to use as output 
explicit_scratch = "C:/temp"
GDB_GUID = ''.join(choice(ascii_uppercase) for i in range(12))
explicit_scratch_ws = arcpy.CreateFileGDB_management(explicit_scratch, "scratch_" + GDB_GUID + ".gdb")
arcpy.env.workspace = os.path.join(explicit_scratch, "scratch_" + GDB_GUID + ".gdb")

failedZoneCheck_tbl = arcpy.management.CreateTable(arcpy.env.workspace, "failedZoneCheck")
arcpy.AddField_management(failedZoneCheck_tbl, "slivers", "TEXT", field_alias="Sliver Polygons", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "toodense", "TEXT", field_alias="Excessive Vertices", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "donuts", "TEXT", field_alias="Interior Voids", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "manyparts", "TEXT", field_alias="Exessive Parts", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "sourceOID", "LONG", field_alias="Source OBJECTID")
arcpy.AddField_management(failedZoneCheck_tbl, "zoneinfo", "TEXT", field_alias="Zone Metadata", field_length = 200)

#Set the fields we will use for the insert cursor
insertFields = ['slivers','toodense','donuts','manyparts','sourceOID','zoneinfo']
updateCur = arcpy.da.InsertCursor(failedZoneCheck_tbl,insertFields)

#We are sorting the zone records by service area name and zone name, which will order them so that duplicate geometry zones are 
#grouped together. We will use a "lastArea" shape.area tracker to recognize when we have encountered a new shape to evaluate - no need to report 
#issues with each identical shape record. 
lastArea = int(0)
print ("Slivers, Excessive Vertices, Interior Voids, Excessive Parts, OID, Shape Specifics") 

#uncomment the version with a whereclause if you want to evaluate a subset of zones - or just set a definition query on the layer
for row in sorted(arcpy.da.SearchCursor(featurelayer, fields)):
#for row in sorted(arcpy.da.SearchCursor(featurelayer, fields, where_clause = expression)):
    shape = row[3]
    thisArea = int(shape.area)
    if (thisArea != lastArea):
        if(shape.length > shape.area):
            sliverShape = True
        else:
            sliverShape = False
        if(shape.length/shape.pointCount <200):
            excessiveVertices = True
        else:
            excessiveVertices = False
        if(shape.boundary().partCount - 1 > shape.partCount):
            interiorRings = True
        else:
            interiorRings = False
        if(shape.partCount > 4):
            manyParts = True
        else:
            manyParts = False
        if(sliverShape or excessiveVertices or interiorRings or manyParts): 
            rowMetadata = "DispLoc={} RouteName={} parts={:d} length={:d} area={:d} point count={:d} ring count={:d}".format(
                                                                                                      row[0],
                                                                                                      row[1],
                                                                                                      shape.partCount, 
                                                                                                      int(shape.length), 
                                                                                                      int(shape.area), shape.pointCount, 
                                                                                                      shape.boundary().partCount)
            
            updateCur.insertRow((str(sliverShape),str(excessiveVertices),str(interiorRings),str(manyParts),row[2],rowMetadata))
            print(str(sliverShape) +  "," + str(excessiveVertices) +  "," + str(interiorRings) +  "," +  str(manyParts) +  "," + rowMetadata)
        
                   
        lastArea = thisArea
del updateCur          

Slivers, Excessive Vertices, Interior Voids, Excessive Parts, OID, Shape Specifics
False,False,True,False,DispLoc=ABI RouteName=ABIL parts=1 length=575791 area=3215823592 point count=659 ring count=27
False,False,True,False,DispLoc=ABI RouteName=BRDY parts=1 length=788711 area=6210198293 point count=903 ring count=33
False,False,True,False,DispLoc=ABI RouteName=BSPG parts=3 length=1347353 area=16040659421 point count=1271 ring count=47
False,False,True,False,DispLoc=ABI RouteName=BWOD parts=1 length=1062583 area=9001979112 point count=1338 ring count=48
False,False,True,False,DispLoc=ABI RouteName=EAST parts=2 length=1411608 area=16884354171 point count=1776 ring count=76
False,False,True,False,DispLoc=ABI RouteName=NORTH parts=2 length=984921 area=11541099092 point count=885 ring count=30
False,False,True,False,DispLoc=ABI RouteName=SANG parts=1 length=1165717 area=12032734768 point count=980 ring count=51
False,False,True,False,DispLoc=ABI RouteName=SONORA parts=3 length=690581 area=

False,False,True,False,DispLoc=AMA RouteName=CITY parts=1 length=528095 area=4889664736 point count=516 ring count=39
False,False,True,False,DispLoc=AMA RouteName=DUMAS parts=2 length=1218140 area=24629983787 point count=614 ring count=32
False,False,True,False,DispLoc=AMA RouteName=ELKCITY parts=4 length=1662302 area=27718431357 point count=1666 ring count=104
False,False,True,False,DispLoc=AMA RouteName=HERFORD parts=3 length=997410 area=15760286884 point count=873 ring count=27
False,False,True,True,DispLoc=AMA RouteName=WEST parts=6 length=2330401 area=34264142663 point count=1851 ring count=96
False,False,True,False,DispLoc=ANC RouteName=CYGAR parts=1 length=107997 area=124026884 point count=237 ring count=11
False,False,True,False,DispLoc=ANC RouteName=FULTN parts=1 length=69929 area=84187126 point count=165 ring count=16
False,False,True,False,DispLoc=ANC RouteName=WANA parts=2 length=76851 area=90298210 point count=196 ring count=15
False,False,True,True,DispLoc=ANC RouteName=W

False,False,True,False,DispLoc=AUS RouteName=D-TOWN parts=1 length=61627 area=80552958 point count=102 ring count=5
False,False,True,False,DispLoc=AUS RouteName=D-TWO parts=2 length=122024 area=156923062 point count=249 ring count=7
False,False,True,False,DispLoc=AUS RouteName=FREDRIC parts=2 length=1553643 area=13079657077 point count=1605 ring count=50
False,False,True,False,DispLoc=AUS RouteName=N-EAST parts=1 length=213885 area=806505638 point count=352 ring count=8
False,False,True,False,DispLoc=AUS RouteName=N-WEST parts=3 length=840692 area=7169071205 point count=1367 ring count=88
False,False,True,False,DispLoc=AUS RouteName=NORTH parts=2 length=282461 area=660077328 point count=649 ring count=42
False,False,True,False,DispLoc=AUS RouteName=NORTH-X parts=1 length=274659 area=788728674 point count=576 ring count=21
False,False,True,False,DispLoc=AUS RouteName=NORTHA parts=2 length=282461 area=660077328 point count=649 ring count=42
False,False,True,False,DispLoc=AUS RouteName=S-

False,False,True,False,DispLoc=BIS RouteName=BISWEST parts=1 length=468673 area=3630098206 point count=523 ring count=3
False,False,True,True,DispLoc=BIS RouteName=DIXN parts=6 length=2443075 area=41685480635 point count=2893 ring count=249
False,False,True,False,DispLoc=BIS RouteName=DTWN parts=1 length=322315 area=1471232787 point count=371 ring count=11
False,False,True,True,DispLoc=BIS RouteName=MGUL parts=15 length=3798564 area=27202799678 point count=3554 ring count=52
False,False,True,True,DispLoc=BIS RouteName=MINOT parts=5 length=2265970 area=26156278620 point count=2322 ring count=111
False,False,True,True,DispLoc=BIS RouteName=MINOT1 parts=8 length=3287330 area=36546395850 point count=3286 ring count=104
False,False,True,True,DispLoc=BIS RouteName=MINOTA parts=5 length=2265970 area=26156278620 point count=2322 ring count=111
False,False,True,False,DispLoc=BKN RouteName=AIRPORT parts=1 length=76049 area=83774646 point count=182 ring count=7
False,False,True,False,DispLoc=BKN 

False,False,True,False,DispLoc=BOS RouteName=POAF parts=4 length=1339807 area=8615119458 point count=1278 ring count=119
False,False,True,False,DispLoc=BOS RouteName=READING parts=1 length=149324 area=406159939 point count=219 ring count=9
False,False,True,False,DispLoc=BOS RouteName=ROCH parts=3 length=247824 area=760871687 point count=304 ring count=10
False,False,True,False,DispLoc=BOS RouteName=SOUTHIE parts=4 length=113972 area=100090738 point count=310 ring count=16
False,False,True,True,DispLoc=BOS RouteName=UNSHORE parts=5 length=460092 area=597147295 point count=1454 ring count=16
False,False,True,False,DispLoc=BOS RouteName=VALLEY parts=1 length=125841 area=404393353 point count=151 ring count=5
False,False,True,False,DispLoc=BOS RouteName=WATER parts=1 length=122753 area=272561649 point count=196 ring count=12
False,False,True,False,DispLoc=BRO RouteName=B-R-O parts=1 length=340711 area=584562219 point count=719 ring count=14
False,False,True,False,DispLoc=BRO RouteName=HRLG

...

False,False,True,False,DispLoc=SPL RouteName=BRUNWIC parts=1 length=118282 area=255379178 point count=225 ring count=13
False,False,True,False,DispLoc=SPL RouteName=CRANBRY parts=1 length=163361 area=442181087 point count=284 ring count=9
False,False,True,False,DispLoc=SPL RouteName=EDISON parts=1 length=73374 area=150985541 point count=128 ring count=5
False,False,True,False,DispLoc=SPL RouteName=FREEHLD parts=2 length=251355 area=947956628 point count=406 ring count=20
False,False,True,False,DispLoc=SPL RouteName=MONMTHA parts=3 length=352744 area=511243796 point count=904 ring count=53
False,False,True,False,DispLoc=SPL RouteName=MONMTHB parts=2 length=160001 area=297695273 point count=341 ring count=15
False,False,True,False,DispLoc=SPL RouteName=MOUNTAN parts=4 length=187484 area=432830854 point count=393 ring count=26
False,False,True,True,DispLoc=SPL RouteName=P-AMBOY parts=5 length=152414 area=134227095 point count=388 ring count=13
False,False,True,False,DispLoc=SPL RouteName=

False,False,True,False,DispLoc=SXF RouteName=MADISON parts=4 length=1263969 area=11074997505 point count=1065 ring count=50
False,False,True,True,DispLoc=SXF RouteName=MINN parts=5 length=1010508 area=7776688346 point count=941 ring count=26
False,False,True,False,DispLoc=SXF RouteName=MINNX parts=4 length=1015951 area=9404268497 point count=1178 ring count=64
False,False,True,False,DispLoc=SXF RouteName=MITCHEL parts=1 length=1229559 area=13499458211 point count=1196 ring count=52
False,False,True,True,DispLoc=SXF RouteName=RUDE parts=6 length=3163073 area=46798748771 point count=2934 ring count=174
False,False,True,False,DispLoc=SXF RouteName=SFEAST parts=1 length=84108 area=149019023 point count=140 ring count=4
False,False,True,False,DispLoc=SXF RouteName=YANKTON parts=2 length=1233029 area=14972965397 point count=1281 ring count=54
False,False,True,False,DispLoc=SYR RouteName=AUBN parts=2 length=753595 area=3727997791 point count=1091 ring count=44
False,False,True,True,DispLoc=SY

False,False,True,True,DispLoc=TPA RouteName=TPA-S parts=8 length=233147 area=166010226 point count=682 ring count=14
False,False,True,False,DispLoc=TPA RouteName=TPA-W parts=2 length=116400 area=171166903 point count=324 ring count=8
False,False,True,False,DispLoc=TUL RouteName=ATOKAX parts=1 length=1008523 area=9571835392 point count=1053 ring count=60
False,False,True,False,DispLoc=TUL RouteName=BVILLEX parts=4 length=1145891 area=11286404889 point count=1554 ring count=94
False,False,True,False,DispLoc=TUL RouteName=DTOWNX parts=1 length=210869 area=760737770 point count=324 ring count=3
False,False,True,False,DispLoc=TUL RouteName=FARSTHX parts=2 length=211425 area=573993390 point count=326 ring count=14
False,False,True,False,DispLoc=TUL RouteName=MCALX parts=4 length=1589400 area=14623666674 point count=1831 ring count=134
False,False,True,False,DispLoc=TUL RouteName=MUSKX parts=4 length=903724 area=7273745035 point count=1215 ring count=86
False,False,True,False,DispLoc=TUL Rout

False,False,True,False,DispLoc=VGS RouteName=NORTH parts=2 length=184966 area=454761001 point count=227 ring count=4
False,False,True,False,DispLoc=VGS RouteName=S-BLVD parts=1 length=76554 area=121509696 point count=135 ring count=4
False,False,True,False,DispLoc=VGS RouteName=SOUTH parts=4 length=116679 area=393034656 point count=167 ring count=11
False,False,True,False,DispLoc=VGS RouteName=WEST parts=2 length=131498 area=285398358 point count=236 ring count=14
False,False,True,False,DispLoc=VRU RouteName=ROOS parts=2 length=1133501 area=19528451778 point count=1126 ring count=79
False,False,True,False,DispLoc=VRU RouteName=VERN parts=3 length=1844130 area=23897751069 point count=1593 ring count=8
False,False,True,False,DispLoc=WAC RouteName=BRYAN parts=1 length=1275049 area=12454898376 point count=2789 ring count=209
False,False,True,False,DispLoc=WAC RouteName=KILLEEN parts=1 length=557698 area=4535009618 point count=916 ring count=35
False,False,True,False,DispLoc=WAC RouteName=N

False,False,True,False,DispLoc=WLP RouteName=VERMONT parts=3 length=590316 area=2773911420 point count=655 ring count=41
False,False,True,False,DispLoc=WOR RouteName=AYER parts=1 length=208523 area=663524552 point count=257 ring count=19
False,False,True,False,DispLoc=WOR RouteName=BOROS parts=1 length=166422 area=566898180 point count=152 ring count=9
False,False,True,False,DispLoc=WOR RouteName=E-CONN parts=1 length=365230 area=2106538568 point count=346 ring count=40
False,False,True,False,DispLoc=WOR RouteName=GARDNER parts=1 length=288820 area=1900167359 point count=146 ring count=16
False,False,True,False,DispLoc=WOR RouteName=MRLBORO parts=1 length=146453 area=457306461 point count=102 ring count=3
False,False,True,False,DispLoc=WOR RouteName=PALMER parts=1 length=490840 area=2437776085 point count=399 ring count=26
False,False,True,False,DispLoc=WOR RouteName=SUTTON parts=1 length=182480 area=787180268 point count=110 ring count=4
False,False,True,False,DispLoc=WOR RouteName=WO